In [1]:
from Logical import LogicalCircuit
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# clearing lib cache during development to force-update imports
import sys, importlib
importlib.reload(sys.modules['Logical'])

<module 'Logical' from '/home/rasmitdevkota/projects/QCA/gatech-qec-project/src/Logical.py'>

In [2]:
stabilizer_tableau = [
  "XXXXIII",
  "IXXIXXI",
  "IIXXIXX",
  "ZZZZIII",
  "IIZZIZZ",
  "IIZZIZZ",
]

In [3]:
# Runs a shot on a noiseless simulator with an error at each of the physical qubits
shots = 1
simulator = AerSimulator()

outputs = []
for i in range(7):
    # Generates the circuit
    lqcirc = LogicalCircuit(2, stabilizer_tableau, name="SteaneCodeCircuit")
    lqcirc.encode(0,1, max_iterations=1, initial_states=[0,1])

    # Inserts an error on both logical qubits
    lqcirc.add_error(0, i, 'X')
    lqcirc.add_error(1, i, 'X')

    # Adds QEC cycle and measurement
    lqcirc.perform_qec_cycle([0, 1])
    lqcirc.measure([0, 1], [0, 1])

    # Simulates the QEC cycle
    qc = transpile(lqcirc, backend=simulator)
    result = simulator.run(qc, shots=shots, memory=True).result()
    output = result.get_memory(lqcirc)
    outputs.append(output[0][:2])

# Outputs should all be 10 if the error correction worked properly
print(outputs)

['10', '10', '10', '10', '10', '10', '10']
